In [6]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from os import listdir

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import os #PythonBib für Datei-Input/-Output

ModuleNotFoundError: No module named 'torch'

In [2]:
import gc

gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('This Conputation is running on {}'.format(device))

NameError: name 'torch' is not defined

Bilder importieren
-

In [ ]:
transforms = transforms.Compose([transforms.Resize(197),
                                 transforms.CenterCrop(197),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))
                                ])

#Batch_size definieren, um Variable wieder zu verwenden und spätere Fehler zu vermeiden
batchSize = 7

#Bilder aus dem verzeichnis in einen Tensor schreiben
true_images_list = []
for f in listdir('C:/Users/meldr/PythonProjects/Aufgabe04/Bilder/cropped/'):
    img = Image.open("C:/Users/meldr/PythonProjects/Aufgabe04/Bilder/cropped/" + f) # f ist gleich der Dateiname
    img_tensor = transforms(img) #(3,100,100)
    true_images_list.append(img_tensor)
train_data = torch.stack(true_images_list) #Liste mit Image-Tensoren

#DataLoader für echte Bilder
trueloader = torch.utils.data.DataLoader(train_data, batch_size=batchSize, shuffle=True, num_workers=1, pin_memory=True) #pin_memory=True -> GPU

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__() #Superkonstruktor für das NN
        #decoder
        #batch_size = 4, size = 2500 (50x50 Pixel)
        self.generate = nn.Sequential(
            nn.ConvTranspose2d(3, 64, 5, stride=2, padding=2), 
            nn.ReLU(), #Relu-Aktivierungsfunktion
            nn.ConvTranspose2d(64, 128, 5, stride=2, padding=2), 
            nn.ReLU(), #Relu-Aktivierungsfunktion
            nn.ConvTranspose2d(128, 3, 5, stride=1, padding=2),
            #Sigmoid-Aktivierungsfunktion, wenn unsere Werte zwischen 0 und 1 sind
            #Tanh-Aktivierungsfunktion, wenn unsere Werte zwischen -1 und 1 sind
            nn.Sigmoid() 
        )
        
    def forward(self,x):
        #print(f'Input Generator:{x.shape}')
        generated = self.generate(x)
        #print(f'Output Generator:{generated.shape}')
        return generated
    


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__() #Superkonstruktor für das NN
        
        #batch_size = 4, size = 2500 (50x50 Pixel)
        self.discriminate = nn.Sequential(
            nn.Conv2d(3, 128, 5, stride=2, padding=2), # 2500=(1+2500-5+2*2)/1
            nn.ReLU(), #Relu-Aktivierungsfunktion
            nn.Conv2d(128, 64, 5, stride=2, padding=2), # 625=(1+2500-5+2*2)/2
            nn.ReLU(), #Relu-Aktivierungsfunktion
            nn.Conv2d(64, 32, 5, stride=2, padding=2), # 125=(1+625-5+2*2)/2
            nn.Sigmoid() 
        )
        
    def forward(self,x):
        #print(f'Input Discriminator:{x.shape}')
        discriminated = self.discriminate(x)
        #print(f'Output Discriminator:{discriminated.shape}')
        return discriminated
    


In [ ]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

criterion = nn.BCELoss().to(device)
gen_optimizer = optim.Adam(generator.parameters(), lr=0.0001)
discr_optimizer = optim.Adam(discriminator.parameters(), lr=0.0001)

In [ ]:

#Definition der Target-Werte, um den Loss zu berechnen
true_target = torch.ones(batchSize, 256, 197, 197, dtype=torch.float).to(device)
#Die echten Inputs sollen auf den Wert 1 trainiert werden
#torch.ones: Gibt einen Tensor zurück, der mit dem Skalarwert 1 gefüllt ist  
#und dessen Form durch das variable Argument size definiert ist. 
#Bspw. torch.ones(5) -> tensor([ 1.,  1.,  1.,  1.,  1.])

false_target = torch.zeros(batchSize,125,3,3, dtype=torch.long).to(device)
#Die falschen Inputs sollen auf en Wert 0 trainiert werden
#torch.zeros: Gibt einen Tensor zurück, der mit dem Skalarwert 0 gefüllt ist  
#und dessen Form durch das variable Argument size definiert ist. 
#Bspw. torch.zeros(5) -> tensor([ 0.,  0.,  0.,  0.,  0.])
fake_image = torch.rand(batchSize,3,50,50).to(device) #torch.Size([3, 200, 200])
for epoch in range(5):
    #cuda-cache leeren
    gc.collect()
    torch.cuda.empty_cache()
    outputs = []
    for true_image in trueloader:
       
        #1.Erstellung einer Random-Noise
        #fake_image = torch.rand(batchSize,3,50,50).to(device) #torch.Size([3, 200, 200])
              
        
        ###### Train Generator
        #Gradienten des Generators zurücksetzen
        gen_optimizer.zero_grad()    
        
        #G.1 Daten mit der Random-Noise generieren
        fake_generated = generator(fake_image)#.to(device)
        #G.2 Fehler mit der fake_generated und den Target 11111... berechnen
        loss_gen_fake = criterion(fake_generated,torch.ones(batchSize, 3, 197, 197, dtype=torch.float).to(device))     
        #G.3 Backpropagation für die fake_generated des Generators        
        loss_gen_fake.backward()
        #G.4 Gewichte des Generators mit dem Gradienten aktualisieren
        gen_optimizer.step()    
        #G.5 Generated Image in outputs speichern
            
            
        ####### Train Discriminator
        #Gradienten des Discriminators zurücksetzen
        discr_optimizer.zero_grad()
    
        #D.1 Train Discriminator mit generierten Fake-Images
        fake_prediction = discriminator(fake_generated)
        #D.2 Fehler mit der fake_prediction und den Target 11111... berechnen
        loss_discr_fake = criterion(fake_prediction.detach(), torch.zeros(batchSize,32, 25, 25, dtype=torch.float).to(device)) 
            #D.3 Backpropagation für die Fake-Prediction des Discriminators
            #loss_discr_fake.backward()
        #D.4 Train Discriminator mit True-Images
        true_prediction = discriminator(true_image.to(device))
        #D.5 Fehler mit der true_prediction und den Target 11111... berechnen
        loss_discr_true = criterion(true_prediction, torch.ones(batchSize,32, 25, 25, dtype=torch.float).to(device))
        #D.6 Backpropagation für die True-Prediction des Discriminators
        loss_discr_true.backward()

        #D.7 Gewichte des Discriminators mit dem Gradienten aktualisieren
        discr_optimizer.step()
        
        outputs.append((epoch, true_image, fake_generated))             


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
for k in range(len(outputs)):
    plt.figure(figsize=(9,2))
    timage = outputs[k*3][1].cpu().detach().numpy()
    fiamge = outputs[k*3][2].cpu().detach().numpy()
    for i, item in enumerate(timage):
        if i >= 9: break
        plt.subplot(2, 9, i+1)
        #item = item.reshape(-1, 28, 28)
        plt.imshow(item[0])
            
    for i, item in enumerate(fiamge):
        if i >= 9:break
        plt.subplot(2,9, 9+i+1)
        #item = item.reshape(-1,28,28)
        plt.imshow(item[0])

Fake-Images generieren
- 

In [ ]:
def random_image():
    rndm_image = np.random.rand(200, 200) #Erstellung eines random arrays mit 200x200 
    rndm_image = Image.fromarray(rndm_image).convert('RGB')  #Array in ein RGB-Array konvertieren
    rndm_image_tensor = transforms(rndm_image) #Array in ein Tensor umwandeln
    return rndm_image_tensor

In [ ]:
fake_images = []
for i in range(338):
    #rndm_image = torch.rand(200,200)[None, :, :]
    rndm_image = np.random.rand(200, 200) #Erstellung eines random arrays mit 200x200 
    rndm_image = Image.fromarray(rndm_image).convert('RGB')  #Array in ein RGB-Array konvertieren
    rndm_image_tensor = transforms(rndm_image) #Array in ein Tensor umwandeln
    fake_images.append(rndm_image_tensor)  #Random Tensor in eine Liste mit Fake Images überführen 
fake_images[0].shape
    
#DataLoader für fake Bilder
fakeloader = torch.utils.data.DataLoader(fake_images, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
f_dataiter = iter(fakeloader)
f_images = f_dataiter.next()

In [ ]:
#Batch_size definieren, um Variable wieder zu verwenden und spätere Fehler zu vermeiden
batchSize = 4

transforms = transforms.Compose([transforms.Resize(50),
                                 transforms.CenterCrop(50),
                                 transforms.ToTensor(),
                                 #transforms.Normalize((0.5,), (0.5,))
                                ])


train_data = torchvision.datasets.ImageFolder('C:/Users/meldr/PythonProjects/Aufgabe04/TeilmengeBilder/', transform=transforms)

#DataLoader für echte Bilder
trueloader = torch.utils.data.DataLoader(train_data, batch_size=batchSize, shuffle=True, num_workers=1) #pin_memory=True -> GPU

In [ ]:
#Wir müssen die Range der Daten analysieren, um zu wissen, welche Aktivierungsfunktion wir verwenden. Die Range kann mittels .Normalize verändert werden
t_dataiter = iter(trueloader)
t_images = t_dataiter.next()
print(torch.min(t_images), torch.max(t_images))

In [ ]:
#outputs = []

#Definition der Target-Werte, um den Loss zu berechnen
true_target = torch.ones(batchSize,3, 50, 50).to(device)
#Die echten Inputs sollen auf den Wert 1 trainiert werden
#torch.ones: Gibt einen Tensor zurück, der mit dem Skalarwert 1 gefüllt ist  
#und dessen Form durch das variable Argument size definiert ist. 
#Bspw. torch.ones(5) -> tensor([ 1.,  1.,  1.,  1.,  1.])

false_target = torch.zeros(batchSize,3,50,50).to(device)
#Die falschen Inputs sollen auf en Wert 0 trainiert werden
#torch.zeros: Gibt einen Tensor zurück, der mit dem Skalarwert 0 gefüllt ist  
#und dessen Form durch das variable Argument size definiert ist. 
#Bspw. torch.zeros(5) -> tensor([ 0.,  0.,  0.,  0.,  0.])

for epoch in range(10):
    for true_img in trueloader:

        ###### Train Generator
        #Gradienten des Generators zurücksetzen
        gen_optimizer.zero_grad()    
        
        #1.Erstellung einer Random-Noise
        fimg = torch.rand(batchSize,3,1,1).to(device) #torch.Size([3, 200, 200])
        
        #2.Generator mittels Random-Noise durchführen 
        fake_gen_out = generator(fimg).to(device)
        print('Generator mit Random-Noise abgeschlossen')
        
        #3.Discriminator mit dem Fake-Output des Generators füttern
        fake_discr_out = discriminator(fake_gen_out)
        print('Discriminator mit generierten Fake-Images abgeschlossen')
    
        #4.Loss für den Fake-Output des Discriminators berechnen
        loss_discr_fake = criterion(fake_discr_out, true_target)
        print('Loss für den Discriminator mit generierten Fake-Output  berechnet')
        
        #5.Backpropagation über den loss des Fake-Outputs des Discriminator
        loss_discr_fake.backward()
        
        #7. Gewichte des Generators mit dem Gradienten aktualisieren
        gen_optimizer.step()
        
        
        
        ####### Train Discriminator
        #Gradienten des Discriminators zurücksetzen
        discr_optimizer.zero_grad()
        
        #1.Discriminator mit den echten Images füttern
        true_discr_out = discriminator(true_img).to(device)
        print('Discriminator mit Echten-Images abgeschlossen')
        print(true_discr_out.shape)
        #print(torch.argmax(true_target, dim=1).shape)
        #2.Loss für den True-Discriminators und den True-Labels bilden
        loss_discr_real = criterion(true_discr_out, true_target)
        print('Loss für den Discriminator mit echten Images berechnet')

        
        #3. Discrimininator mit den Fake-Output des Generators füttern
        gen_discr_out = discriminator(fake_discr_out.detach()).to(device)
                #.detach() Erzeugt einen Tensor, der sich den Speicher mit einem 
                #Tensor teilt, der keinen Gradient benötigt. Er trennt die Ausgabe
                #vom Berechnungsgraphen ab. Es wird also kein Gradient entlang dieser
                #Variablen zurückverfolgt.
       # print('Discriminator mit Echten-Images abgeschlossen')

        
        #4. Loss für den Fake-Output des Discriminators berechnen
        gen_discr_loss = criterion(gen_discr_out, false_target)
                
                
        #5. Finalen loss mitteln 
        discr_loss = (loss_discr_real + gen_discr_loss)  / 2
        
        #6. Backpropagation
        discr_loss.backward()
        
        #7. Gewichte des Discriminators mit dem Gradienten aktualisieren
        discr_optimizer.step()
        
    
        


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__() #Superkonstruktor für das NN
        
        #batch_size = 4, size = 2500 (50x50 Pixel)
        self.conv1 = nn.Conv2d(3, 250, 5, stride=1, padding=2), # 250 = (1+250-5+2*2)/1
        self.conv1ReL = nn.ReLU(), #Relu-Aktivierungsfunktion
        self.conv2 = nn.Conv2d(250, 125, 5, stride=2, padding=2), #1250 = (1+250-5+2*2)/2
        self.conv2ReL = nn.ReLU(), #Relu-Aktivierungsfunktion
        self.conv2 = nn.Conv2d(125, 31, 6, stride=4, padding=2), # 31 = (1+125-6+2*2)/4
       
        self.fc1 = nn.Linear(16*5*5,200)
        self.fc2 = nn.Linear(200,150)
        self.fc3 = nn.Linear(150,10)
   
        
    def forward(self,x):
        
        print(f'Input Discriminator:{x.shape}')
        x = self.conv1ReL(self.conv1(x))
        x = self.conv2ReL(self.conv2(x))
        x = x.view(-1,77500)
        print(f'Output Discriminator:{x.shape}')
        return x
    
